In [1]:
import json

f = open('/home/doruk/Storage/bioComputing/go/go.json', 'r')
go = json.load(f)

In [6]:
def namespace(uri:str) -> str:
    nsdict = {
        'http://purl.obolibrary.org/obo/go.owl#': 'go_owl',
        'http://purl.obolibrary.org/obo/GO_': 'GO'
    }
    for k,v in nsdict.items():
        if uri.startswith(k):
            return f'{v}:{uri.replace(k,"")}'
    return uri

In [7]:
import pandas as pd 

nodeList = []
for i in go['graphs'][0]['nodes']:
    n = {}
    if i['type'] == 'CLASS':
        n['id'] = namespace(i['id'])
        if i['meta']['definition'] is not None:
            n['definition'] = i['meta']['definition']['val']
        nodeList.append(n)
nodes = pd.DataFrame(nodeList)
nodes

,id,definition
0,GO:0000001,"The distribution of mitochondria, including th..."
1,GO:0000002,The maintenance of the structure and integrity...
2,GO:0000003,The production of new individuals that contain...
3,GO:0000004,NaN
4,GO:0000005,OBSOLETE. Assists in the correct assembly of r...
...,...,...
49826,GO:2001313,The chemical reactions and pathways involving ...
49827,GO:2001314,The chemical reactions and pathways resulting ...
49828,GO:2001315,The chemical reactions and pathways resulting ...
49829,GO:2001316,The chemical reactions and pathways involving ...


In [9]:
trplist = []
for i in go['graphs'][0]['edges']:
    t = {}
    t['s'] = namespace(i['sub'])
    t['p'] = namespace(i['pred'])
    t['o'] = namespace(i['obj'])
    trplist.append(t)
tripleStore = pd.DataFrame(trplist)
tripleStore

,s,p,o
0,GO:0000001,is_a,GO:0048308
1,GO:0000001,is_a,GO:0048311
2,GO:0000002,is_a,GO:0007005
3,GO:0000003,is_a,GO:0008150
4,GO:0000006,is_a,GO:0005385
...,...,...,...
95738,GO:2001317,is_a,GO:1901362
95739,GO:2001317,is_a,GO:2001316
95740,go_owl:happens_during,is_a,go_owl:ends_during
95741,go_owl:negatively_regulates,is_a,go_owl:regulates


In [10]:
tripleStore.to_csv('edges.csv', sep='\t')
nodes.to_csv('nodes.csv', sep='\t')

In [12]:
noi = 'GO:0007295'

sub = tripleStore[tripleStore['s'] == noi]
display(sub)
print(sub['o'].tolist())

,s,p,o
10033,GO:0007295,is_a,GO:0003006
10034,GO:0007295,is_a,GO:0048589
10035,GO:0007295,go_owl:part_of,GO:0048477


['GO:0003006', 'GO:0048589', 'GO:0048477']


In [17]:
subtrpls = []

def traverseUp(triples:pd.DataFrame, noil:list):
    for noi in noil:
        print(noi)
        subjdf = triples[triples['s'] == noi]
        print(subjdf)
        for i,r in subjdf.iterrows():
            t = {
                    's': r['s'], 
                    'p': r['p'],
                    'o': r['o']
                }
            if findspo(t) == -1:
                subtrpls.append(t)
        traverseUp(triples=triples, noil=subjdf['o'].tolist())

def traverseDown(edgedf:pd.DataFrame, noil:list):
    for noi in noil:
        objdf = edgedf[edgedf['o'] == noi]
        for i,r in objdf.iterrows():
            t = {
                    's': r['s'], 
                    'p': r['p'],
                    'o': r['o']
                }
            if findspo(t) == -1:
                subtrpls.append(t)
        traverseDown(edgedf=edgedf, noil=objdf['s'].tolist())

def findspo(spo:dict) -> int:
    for i in range(len(subtrpls)):
        if spo == subtrpls[i]:
            print(i)
            return i
    return -1

In [18]:
traverseUp(tripleStore, [noi])
traverseDown(tripleStore, [noi])
subdf = pd.DataFrame(subtrpls)
subdf

GO:0007295
                s               p           o
10033  GO:0007295            is_a  GO:0003006
10034  GO:0007295            is_a  GO:0048589
10035  GO:0007295  go_owl:part_of  GO:0048477
GO:0003006
               s     p           o
4517  GO:0003006  is_a  GO:0022414
4518  GO:0003006  is_a  GO:0032502
GO:0022414
                s               p           o
23459  GO:0022414            is_a  GO:0008150
23460  GO:0022414  go_owl:part_of  GO:0000003
GO:0008150
Empty DataFrame
Columns: [s, p, o]
Index: []
GO:0000003
            s     p           o
3  GO:0000003  is_a  GO:0008150
GO:0008150
Empty DataFrame
Columns: [s, p, o]
Index: []
GO:0032502
                s     p           o
28219  GO:0032502  is_a  GO:0008150
GO:0008150
Empty DataFrame
Columns: [s, p, o]
Index: []
GO:0048589
                s     p           o
48086  GO:0048589  is_a  GO:0032502
48087  GO:0048589  is_a  GO:0040007
GO:0032502
                s     p           o
28219  GO:0032502  is_a  GO:0008150
8
GO:0008150

,s,p,o
0,GO:0007295,is_a,GO:0003006
1,GO:0007295,is_a,GO:0048589
2,GO:0007295,go_owl:part_of,GO:0048477
3,GO:0003006,is_a,GO:0022414
4,GO:0003006,is_a,GO:0032502
5,GO:0022414,is_a,GO:0008150
6,GO:0022414,go_owl:part_of,GO:0000003
7,GO:0000003,is_a,GO:0008150
8,GO:0032502,is_a,GO:0008150
9,GO:0048589,is_a,GO:0032502


In [15]:
import pygraphviz as pgv

g = pgv.AGraph(directed=True)
for i,r in subdf.iterrows():
    s = str(r['s'])
    p = str(r['p'])
    o = str(r['o'])
    if s not in g:
        g.add_node(s, label=s)
    if o not in g:
        g.add_node(o, label=o)
    g.add_edge(s, o, label=p)
g.layout(prog='dot')
g.draw('go3.png')